# [Snow monitoring with Landsat](https://docs.dea.ga.gov.au/notebooks/05_Temporal_analysis/SnowMonitoringUsingLandsat.html)

## What does this notebook do?

This notebook gets the Landsat 8 data that has 6 spectral bands including visible (red,green,blue) and invisible (nir, swir1, swir2) bands. These bands are plotted individually and combined. Also, the NDVI and NDWI are calculated and plotted from these bands.

Since the production GSKY server can only return three bands at a time, two separate calls to GSKY are made to get all six bands.

###### Specifics

- Converting a DEA notebook example to use GSKY instead of datacube.

- Uses WCS to get an area from a specified layer/date/bbox/time, and differs from the DEA example.

- Only one time slice can be used in WCS. 
    - You can choose from all available times in the layer.

- If you can determine a valid BBox for a date (Tip: Use TerriaMap) it can be manually typed in.

- The DEA example, using datacube, can get the data for a time range. It is not possible in GSKY.

**Author:** Arapaut V. Sivaprasad

**Date:** 25 October, 2019

In [1]:
##########################################################
# Author: Arapaut V. Sivaprasad.                         #
# Dates: Created: 22/10/2019; Last Modified: 15/01/2020  #
##########################################################

from owslib.wcs import WebCoverageService
import matplotlib.pylab as plt
%matplotlib inline
from netCDF4 import Dataset
import numpy as np
import logging, sys
logging.disable(sys.maxsize)


### The WCS service on the GSKY server

The GSKY server is publicly available and runs on one of NCI's HPC nodes. To get the data a regular HTTPS call is made with the required parameters as a GET command.

The WCS (Web Coverage Service) is a very efficient way to get data spread across several hundred to thousands of files and directories. By specifying an area, date and data layer name the WCS service retrieves all the required data and combines it into a single NetCDF (or GeoTIFF) output. In this example we will use the NetCDF format to get the data and then plol it and/or analyse the data to pick up other features.

In [2]:
# Specify the GSKY production URL here. The layer is hard coded
gsky_url = 'https://gsky.nci.org.au/ows/dea'
wcs = WebCoverageService(gsky_url, version='1.0.0')
layer = "landsat8_nbar_16day"

# Uncomment below to see all layers.

#for layer in list(wcs.contents):
#    print ("Layer Name:", layer)
#    print ("Title:", wcs[layer].title, '\n')

### Get a time stamp

The data is stored as separate files with a time stamp, or time position, in each. This time stamp correspond to the data acqusition date and is used to retrieve the data for a particular area. Only one time stamp can be specified in one WCS query.

In [3]:
# To display the available time slices in any range, specify the start and end values as m,n below
time = wcs[layer].timepositions
n_times = len(time)
m = 0
n = n_times-1
#print ("Total of", n_times,'timepositions:\n\n', time[m:n])

# Unlike in the case of datacube, where a time range can be specified, GSKY requires a specific date.
# This specific time slice is specified below. 
t = 1
wcs[layer].timepositions[t]

'2013-04-04T00:00:00.000Z'

### Specify a bounding box

To specify an area to analyse, you must give its bounding box coordinates (in EPSG:4326). This can be done either by hard coding it as e.g. *BBoxes[0] = ['148.62,-35.14,149.32,-36.58']* or by giving a tile ID(s) to calculate the BBox. Only one BBox can be specified in a WCS query.

The tiles are approximately 1x1 degree areas the Australian continent is divided into. A list of these tiles and their bounding boxes can be seen by uncommenting its code below. For instance, tiles '15_-40' and '15_-41' together cover all of ACT plus some of the surroundings. Sydney is covered by '17_-39'. 

Click [here](http://cmi.ga.gov.au/sites/default/files/2017-12/DEA%20Tiles_1.pdf) to view the tile map.


In [4]:
tiles_list = {}
with open("Scripts/tiles_bboxes_out.txt") as f:
    for line in f:
        (key, val) = line.split()
        tiles_list[str(key)] = val

# Uncomment below to see all tiles and ther BBoxes
#tiles_list

#### Select a tile(s)

Give a tile ID to select an area. To specify an area larger than one tile you can give the top left corner and the bottom right corner tiles. A bounding box covering the entire area will be calculated. Please be aware that too large an area will cause a timeout error at the GSKY server.

In [5]:
# Give one tile or two tiles as [top left corner, bottom right corner] below.
#tiles = ['17_-39']  # Sydney
tiles = ['15_-40','15_-41']  # ACT
bbox = [tiles_list[tiles[0]],tiles_list[tiles[1]]]

# If you know a BBox containing data (Tip: use TerriaMap to find out) then type it in below.
#bbox = ['150.52,-34.15,151.34,-33.56'] # Sydney
bbox

['148.50,-34.71,149.73,-35.72', '148.64,-35.60,149.89,-36.61']

### Calculate the bounding box and adjust it to fit the plotting area

The plotting area has a default dimension of 337x240 pixels. When the area of the bounding box is different to this, the image will be adjusted to fit in this plot. It can result in distortion of the image. To avoid it, we re-calculate the bounding box below.

In [6]:
x = float(bbox[0][0:6])
y = float(bbox[0][7:13])
X = float(bbox[1][14:20])
Y = float(bbox[1][21:27])

# The longitude dimension is adjusted to fit the plot size of 337*220 pixels. 
# It is required to avoid distorting the image
# X = float('%.2f'%(x + ((abs(Y - y) / 220) * 337)))
bbox = (x,y,X,Y)
bbox

(148.5, -34.71, 149.89, -36.61)

### Get the data from GSKY server using the WCS service

The call below asks for the True Color bands (red, green, blue) and False Color bands (nir, swir1, swir2). GSKY always labels the bands as Band 1, 2 and 3. These are taken into variables of appropriate names.

In [8]:
# Construct the URL for a WCS call and send it to the GSKY server.
# Style can be False Color (fc) or True Color (tc)
# This NB uses only NetCDF and EPSG:4326. Hence, do not change these.
# If any error, the displayed URL can be opened in a browser to see the actual error message.
output = wcs.getCoverage(identifier=layer, Styles='tc',
                         time=[wcs[layer].timepositions[t]],
                         bbox=bbox,format='NetCDF',
                         crs='EPSG:4326', width=256, height=256)
# Read the output into diskless NetCDF
nc = Dataset('dtemp.nc','r',diskless=True,persist=False, memory=output.read())
r = nc.variables['Band1'][:]
g = nc.variables['Band2'][:]
b = nc.variables['Band3'][:]

# print(output.geturl())

output = wcs.getCoverage(identifier=layer, Styles='fc',
                         time=[wcs[layer].timepositions[t]],
                         bbox=bbox,format='NetCDF',
                         crs='EPSG:4326', width=256, height=256)
# Read the output into diskless NetCDF
nc = Dataset('dtemp.nc','r',diskless=True,persist=False, memory=output.read())
s1 = nc.variables['Band1'][:]
n = nc.variables['Band2'][:]
g = nc.variables['Band3'][:]

# print(output.geturl())

HTTPError: 500 Server Error: Internal Server Error for url: http://gsky.nci.org.au/ows/dea?version=1.0.0&request=GetCoverage&service=WCS&Coverage=landsat8_nbar_16day&BBox=148.5%2C-34.71%2C149.89%2C-36.61&time=2013-04-04T00%3A00%3A00.000Z&crs=EPSG%3A4326&format=NetCDF&width=256&height=256&Styles=tc

### Plot the data

The plots below will display, singly or together, the true color and false color data.

In [ ]:
plt.imshow(b, cmap='Blues')
plt.title('Blue')
plt.colorbar()
plt.show

In [ ]:
# Plot the green band from true color
plt.imshow(g, cmap='Greens')
plt.title('Green')
plt.colorbar()
plt.show

In [ ]:
# Plot the red band  from true color
plt.imshow(r, cmap='Reds')
plt.title('Red')
plt.colorbar()
plt.show

In [ ]:
# Plot all three true color bands together
# Stack the three bands together and cast it as float32
# In the line below the 'axis=2' ensures that the data shape is 256,256,3
# rgb = np.stack([nc.variables['Band%d' % (b+1)] for b in range(3)], axis=2).astype(np.float32)
rgb = np.stack([r, g, b], axis=2).astype(np.float32)
# Convert all values into a range from 0 to 1. This is important.
rgb /= rgb.max()

# Uncomment next line to see the data values.
# print(rgb)

# Plot it
imgplot = plt.imshow(rgb)
plt.show()

In [ ]:
# Plot the swir1
plt.imshow(s1, cmap='gist_rainbow')
plt.title('Swir1')
plt.colorbar()
plt.show

In [ ]:
# Plot the nir
plt.imshow(n, cmap='Accent')
plt.title('Nir')
plt.colorbar()
plt.show

In [ ]:
# Plot the green
plt.imshow(g, cmap='Greens')
plt.title('Green')
plt.colorbar()
plt.show

In [ ]:
# Plot all three False color bands together
# Stack the three bands together and cast it as float32
# In the line below the 'axis=2' ensures that the data shape is 256,256,3
# rgb = np.stack([nc.variables['Band%d' % (b+1)] for b in range(3)], axis=2).astype(np.float32)
rgb = np.stack([s1, n, g], axis=2).astype(np.float32)
# Convert all values into a range from 0 to 1. This is important.
rgb /= rgb.max()

# Uncomment next line to see the data values.
# print(rgb)

# Plot it
imgplot = plt.imshow(rgb)
plt.show()

### Compute Normalised Difference Water Index (NDWI)

The NDWI index is most appropriate for water body mapping. The water body has strong absorbability and low radiation in the range from visible to infrared wavelengths. The index uses the green and Near Infra-red bands of remote sensing images based on this phenomenon.

In [ ]:
# Compute NDWI using the formula (green - nir) / (green + nir). This will calculate 
# NDWI for every time-step in the dataset:
ds_ndwi = (g - n) / (g + n)

# Uncomment below to see the data
# ds_ndwi

plt.imshow(ds_ndwi, cmap='Spectral')
plt.title('ds_ndwi')
plt.colorbar()
plt.show

### Compute Normalised Difference Vegetation Index (NDVI)¶

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements and assess whether the target contains live green vegetation or not.

In [ ]:
# Compute NDVI using the formula (nir - red) / (nir + red)
ds_ndvi = (n - r) / (n + r)

# Plot:
plt.imshow(ds_ndvi, cmap='winter')
plt.title('ds_ndvi')
plt.colorbar()
plt.show

### Compute Normalised Difference Snow Index (NDSI)

Normalized Difference Snow Index (NDSI) is used to delineate the presence of snow/ice. It is a standardized ratio of the difference in the reflectance in the bands that take advantage of unique signature and the spectral difference to indicate snow from the surrounding features and even clouds.

In [ ]:
# Compute index by subtracting, summing and dividing the `green` and `swir1` bands
ds_ndsi = (g - s1) / (g + s1)

# Plot one sample timestep from the NDSI dataset
plt.imshow(ds_ndsi, cmap='cool')
plt.title('ds_ndsi')
plt.colorbar()
plt.show

### Threshold NDSI and NDVI to identify snowy, non-vegetated pixels



In [ ]:
# Threshold that will be applied to the NDSI layer to identify pixels containing snow
ndsi_threshold = 0

# Threshold that will be applied to the NDVI layer to identify pixels containg vegetation
ndvi_threshold = 0.3

# Apply NDSI threshold defined above to identify snow (pixels with NDSI values greater than X)
snow_ds = ds_ndsi > ndsi_threshold

# Apply NDSI threshold defined above to identify snow (pixels with NDVI values less than X)
noveg_ds = ds_ndvi < ndvi_threshold

# Finally, keep only pixels that are both snowy AND non-vegetated:
snow_ds = noveg_ds & noveg_ds
# Plot the resulting layer
plt.imshow(snow_ds, cmap='BuPu')
plt.title('snow_ds')
plt.show

`Copyright` &copy; `2019 by National Computational Infrastructure (NCI).`